<a href="https://colab.research.google.com/github/quinnie-nguyen/WiDS2021/blob/main/Shopee_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Assuming that a customer can use multiple email addresses and phone numbers to contact customer service regarding there orders (several order ID).

Main Task:



*   Identify the number of unique customers in data set
*   Investigate which tickets belong to a customer



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# import libraries
import numpy as np
import pandas as pd
import json

In [4]:
# load data
df = open('/content/gdrive/My Drive/Shopee/contacts.json')
data = json.load(df)
df = pd.DataFrame.from_dict(data)
df.tail()

,Id,Email,Phone,Contacts,OrderId
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku
499999,499999,,880053388839,0,JibSBRgzYdfzkzbTuGUXrcvDX


In [5]:
group = pd.DataFrame(df.groupby(['Email','Phone','OrderId', 'Id'])['Contacts'].sum())
group.reset_index(inplace = True)
group = group.sort_values(by = 'Id', ascending= True)

In [6]:
group.head()

,Email,Phone,OrderId,Id,Contacts
370522,gkzAbIy@qq.com,,,0,1
50247,,329442681752,vDDJJcxfLtSfkooPhbYnJdxov,1,4
138162,,9125983679,,2,0
410085,mdllpYmE@gmail.com,,bHquEnCbbsGLqllwryxPsNOxa,3,0
45812,,300364407,,4,2


# Emails

In [ ]:
# remove rows that email is null
group_Email=group[~group["Email"].eq('')]
group_Email.shape

(348606, 5)

In [ ]:
# group by Email, join the ID by '-'
group_Email['by_email'] = group_Email[['Email','Id']].groupby(['Email'])['Id'].transform(lambda x: '-'.join(map(str, x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
group_Email=group_Email[['Id','by_email']]

# Order IDs

In [ ]:
group_Oder=group[~group["OrderId"].eq('')]
group_Oder.shape

(243544, 5)

In [ ]:
group_Oder['by_OrderID'] = group_Oder.groupby(['OrderId'])['Id'].transform(lambda x: '-'.join(map(str, x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
group_Oder=group_Oder[['Id','by_OrderID']]

# Phones

In [ ]:
group_Phone=group[~group["Phone"].eq('')]
group_Phone.shape

(245081, 5)

In [ ]:
group_Phone['by_Phone'] = group_Phone.groupby(['Phone'])['Id'].transform(lambda x: '-'.join(map(str, x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
group_Phone=group_Phone[['Id','by_Phone']]

# Concate Infos

In [ ]:
# concate the three info types
group1=group.merge(group_Email, on='Id')
group2=group1.merge(group_Oder, on='Id')
group3=group2.merge(group_Phone, on='Id')

In [ ]:
group3.head(10)

,Email,Phone,OrderId,Id,Contacts,by_email,by_OrderID,by_Phone
0,YsKhkKGJoQSNniHcr@yahoo.com,562300766,IqewrMXVTJqSsVklWFfOpdRNI,13,1,13-417916,9-13-343161,13-16708
1,TNaMorNIWCJARsXlg@gmail.com,092607900110,GqpDeqZSwZTbPRrrGePkOhbSQ,17,3,17,17,15-17-21197
2,FoaGWGbpnlvizkIG@qq.com,87750412055,FAfkaPfbiYqJNLqonkuozigbW,22,0,22-453177,22,22-4562
3,MbjTCmxpcqf@yahoo.com,028044435230,nfVgFexPZGijuRCTKQOCRYKBe,28,1,28,28-1271-26068,28
4,VbaKKxkCzo@yahoo.com,315194406,TsBwFmhSCeauMjfoMVkhTBdYI,42,3,42-164913,42,42
5,TIMejlVYhfqjy@gmail.com,840113148,XBkNrwcWWslDvSQdhYPfCpQJo,50,4,50,50-226720-458692,5-50
6,LEQvQVjRnZGUzzii@qq.com,677071683,tNyZAarWISkKcyhLPjYOVXfrD,51,4,51,51,35-51
7,MqeWqLfAIPvgaelFet@hotmail.com,173754467363,yszJKOtWYrnjJoMPmXQMuZhiu,68,3,68-187301,68,68-1321-321667
8,KVJqkaaCFmoNXwysz@qq.com,075673879,RKKKiOTjYrrtRjRqtjzxuMnag,100,2,100,100-822-282502-306045,100-79530-220328
9,ckULfmuWRNSEE@gmail.com,67297509936,iqSSKRKJuXKNTKCUPwTrencOp,109,3,109,109,109-294-237280


In [ ]:
# combine 3 ticket traces grouped by email, orderID, phone
group3['group']=group3['by_email']+"-"+group3['by_OrderID']+"-"+group3['by_Phone']
group3=group3[['Email','Phone','OrderId','Id','Contacts','group']]

In [ ]:
group3.head(10)

,Email,Phone,OrderId,Id,Contacts,group
0,YsKhkKGJoQSNniHcr@yahoo.com,562300766,IqewrMXVTJqSsVklWFfOpdRNI,13,1,13-417916-9-13-343161-13-16708
1,TNaMorNIWCJARsXlg@gmail.com,092607900110,GqpDeqZSwZTbPRrrGePkOhbSQ,17,3,17-17-15-17-21197
2,FoaGWGbpnlvizkIG@qq.com,87750412055,FAfkaPfbiYqJNLqonkuozigbW,22,0,22-453177-22-22-4562
3,MbjTCmxpcqf@yahoo.com,028044435230,nfVgFexPZGijuRCTKQOCRYKBe,28,1,28-28-1271-26068-28
4,VbaKKxkCzo@yahoo.com,315194406,TsBwFmhSCeauMjfoMVkhTBdYI,42,3,42-164913-42-42
5,TIMejlVYhfqjy@gmail.com,840113148,XBkNrwcWWslDvSQdhYPfCpQJo,50,4,50-50-226720-458692-5-50
6,LEQvQVjRnZGUzzii@qq.com,677071683,tNyZAarWISkKcyhLPjYOVXfrD,51,4,51-51-35-51
7,MqeWqLfAIPvgaelFet@hotmail.com,173754467363,yszJKOtWYrnjJoMPmXQMuZhiu,68,3,68-187301-68-68-1321-321667
8,KVJqkaaCFmoNXwysz@qq.com,075673879,RKKKiOTjYrrtRjRqtjzxuMnag,100,2,100-100-822-282502-306045-100-79530-220328
9,ckULfmuWRNSEE@gmail.com,67297509936,iqSSKRKJuXKNTKCUPwTrencOp,109,3,109-109-109-294-237280


In [ ]:
# transform the string to a list of elements of strings
split_group = group3['group'].str.split('-')

split_group = split_group.to_list()


In [ ]:
# change type of elements of lists (string --> integer)
for i in range(0,len(split_group)):
  for j in range(0,len(split_group[i])):
    split_group[i][j] = int(split_group[i][j])

In [ ]:
list(set(split_group[1]))

[17, 21197, 15]

In [ ]:
# eliminate the duplicates in list
for i in range(0, len(split_group)):
  split_group[i] = list(set(split_group[i]))

In [ ]:
# sort the integers in list in ascending order
for i in range(0, len(split_group)):
  split_group[i] = sorted(split_group[i])

In [ ]:
split_group[1:10]

['15-17-21197',
 '22-4562-453177',
 '28-1271-26068',
 '42-164913',
 '5-50-226720-458692',
 '35-51',
 '68-1321-187301-321667',
 '100-822-79530-220328-282502-306045',
 '109-294-237280']

In [ ]:
# integer --> string
for i in range(0,len(split_group)):
  for j in range(0,len(split_group[i])):
    split_group[i][j] = str(split_group[i][j])
# join the elements by '-'
for i in range(0,len(split_group)):
  split_group[i] = '-'.join(split_group[i])

In [ ]:
group3['ticket_trace'] = split_group

In [ ]:
group3.head()

,Email,Phone,OrderId,Id,Contacts,group,ticket_trace
0,YsKhkKGJoQSNniHcr@yahoo.com,562300766,IqewrMXVTJqSsVklWFfOpdRNI,13,1,13-417916-9-13-343161-13-16708,9-13-16708-343161-417916
1,TNaMorNIWCJARsXlg@gmail.com,092607900110,GqpDeqZSwZTbPRrrGePkOhbSQ,17,3,17-17-15-17-21197,15-17-21197
2,FoaGWGbpnlvizkIG@qq.com,87750412055,FAfkaPfbiYqJNLqonkuozigbW,22,0,22-453177-22-22-4562,22-4562-453177
3,MbjTCmxpcqf@yahoo.com,028044435230,nfVgFexPZGijuRCTKQOCRYKBe,28,1,28-28-1271-26068-28,28-1271-26068
4,VbaKKxkCzo@yahoo.com,315194406,TsBwFmhSCeauMjfoMVkhTBdYI,42,3,42-164913-42-42,42-164913


In [ ]:
group3.shape[0]

40326

# Conclusion:

We are asked to work with the massive data set (500000 rows). This task could be done easily using LOOP-command. However, it is infeasible doing so for massive data set.

This time we applied the GROUP BY, and buid-in functions for string.

Participants:


1.   Ngoc Quyen Nguyen - M.Sc Data Science (Heidelberg University)
2.   Ngoc Chien  Le - M.Sc Data Science (Australia University)
3.   Thanh Quyen Nguyen - M.Sc Econometrics (TU Dortmund)
4.   Hoang Phuc Nguyen - M.Sc Computer Science (Ho Chi Minh City University of Technology)


